In [1]:
import pandas as pd 
import pymysql
from sqlalchemy import create_engine

In [3]:
df = pd.read_csv('University_cleaned.csv', encoding='latin1')

In [4]:
df.head(5)

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University


In [6]:
conn = pymysql.connect(
    host = 'localhost',
    port = 3307,
    password = 'mGFeanZG10784',
    user = 'amdariuser',
    database = 'amdaridb'
)

In [7]:
cursor = conn.cursor()

In [33]:
cursor.execute("SHOW DATABASES;")

3

In [34]:
cursor.fetchall()

(('amdaridb',), ('information_schema',), ('performance_schema',))

In [35]:
create_query_file = open('./sql/mysql_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university(\n    id BIGINT AUTO_INCREMENT NOT NULL,\n    country VARCHAR(200) NOT NULL,\n    `name` VARCHAR(200) NOT NULL,\n    founded INT NOT NULL,\n    `type` VARCHAR(200) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(200) NOT NULL,\n\n    UNIQUE KEY unique_combination(`country`,`name`),\n    PRIMARY KEY(id)\n);\n\n    \n'

In [36]:
cursor.execute(create_query)

0

In [56]:
data = df.itertuples(index=None,name=None)

In [57]:
data

In [58]:
data2 = list(tuple(data))

In [59]:
data2

[('Albania',
  'University of Tirana',
  1957,
  'Public',
  35000,
  'https://en.wikipedia.org/wiki/University_of_Tirana'),
 ('Algeria',
  'Constantine University',
  1978,
  'Public',
  85000,
  'https://en.wikipedia.org/wiki/List_of_universities_in_Algeria'),
 ('Angola',
  'Agostinho Neto University',
  1962,
  'Public',
  29827,
  'https://en.wikipedia.org/wiki/Agostinho_Neto_University'),
 ('Argentina',
  'University of Buenos Aires',
  1821,
  'Public',
  311175,
  'https://en.wikipedia.org/wiki/University_of_Buenos_Aires'),
 ('Australia',
  'Monash University',
  1958,
  'Public',
  73807,
  'https://en.wikipedia.org/wiki/Monash_University'),
 ('Austria',
  'University of Vienna',
  1365,
  'Public',
  91000,
  'https://en.wikipedia.org/wiki/University_of_Vienna'),
 ('Bangladesh',
  'National University, Bangladesh',
  1992,
  'Public',
  2097182,
  'https://en.wikipedia.org/wiki/National_University,_Bangladesh'),
 ('Belarus',
  'Belarusian State University',
  1921,
  'Public',

In [25]:
engine = create_engine("mysql+pymysql://", creator=lambda:conn)

In [37]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link


In [63]:
merge_query_file = open('./sql/mysql_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'INSERT INTO university(country,name,founded,type,enrollment,link)\nVALUES(%s,%s,%s,%s,%s,%s)\nON DUPLICATE KEY UPDATE\n    founded=VALUES(founded),\n    `type`=VALUES(type),\n    enrollment=VALUES(enrollment),\n    link=VALUES(link);'

In [64]:
cursor.executemany(merge_query,data2)

70

In [65]:
conn.commit()

In [66]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link
0,1,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,2,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,3,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,4,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,5,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
65,66,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
66,67,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
67,68,United States,Texas A&M University,1876,Public,73284,https://en.wikipedia.org/wiki/Texas_A%26M_Univ...
68,69,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...
